# Generate report from exprted photometer data  

Use raw folder with photometer exported data

## Imports

In [1]:
VERBOSE_NOTEBOOK = False
WARNING_DISABLE = True
DEBUG = False

In [2]:
from os import path
import warnings
from scipy.optimize import OptimizeWarning

if WARNING_DISABLE:
    warnings.simplefilter('ignore', RuntimeWarning)
    warnings.simplefilter('ignore', OptimizeWarning)
    warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [3]:
from hamrep.mkinout import make_input_paths, parse_dir_name

WORKING_DIR = './reports/export/230801_AAV9-ELISA_sey_GN004240-053'
DATA_DIR = './data'

PARSED_DIR = parse_dir_name(WORKING_DIR)

input_files = make_input_paths(WORKING_DIR)
WORKLIST_FILE_PATH = input_files['worklist']
PARAMS_FILE_PATH = input_files['params']

In [4]:
from dataclasses import dataclass
import json

@dataclass
class Config:
    def __init__(self, 
        pandoc_bin: path,
        pdflatex_bin: path,
        reference_docx: path,
        data_dir: path = "./data",
        params_filename: path = "./data/params.json",
        plate_layout_id: path = "./data/plate_layout_ident.csv",
        plate_layout_num: path = "./data/plate_layout_num.csv",
        plate_layout_dil_id: path = "./data/plate_layout_dil_id.csv",
        numeric_warning_disable: bool = True 
    ):
        self.pandoc_bin = pandoc_bin
        self.pdflatex_bin = pdflatex_bin
        self.reference_docx = reference_docx
        self.data_dir = data_dir
        self.params_filename = params_filename
        self.plate_layout_id = plate_layout_id
        self.plate_layout_num = plate_layout_num
        self.plate_layout_dil_id = plate_layout_dil_id
        self.numeric_warning_disable = numeric_warning_disable

def read_config(cfg_path):
    with open(path.join(DATA_DIR, "config.json")) as json_file:
        jd = json.load(json_file)

    return Config(jd['pandoc_bin'], jd['pdflatex_bin'], jd['reference_docx'])

cfg = read_config('./data/config.json')
cfg.params_filename

'./data/params.json'

## Layouts

In [5]:
from hamrep.readdata import read_layouts

PLATE_LAYOUT_ID = 'plate_layout_ident.csv'
PLATE_LAYOUT_NUM = 'plate_layout_num.csv'
PLATE_LAYOUT_DIL_ID = 'plate_layout_dil_id.csv'


g_lay = read_layouts(path.join(DATA_DIR, PLATE_LAYOUT_ID),
                     path.join(DATA_DIR, PLATE_LAYOUT_NUM),
                     path.join(DATA_DIR, PLATE_LAYOUT_DIL_ID))

if VERBOSE_NOTEBOOK:
    display(g_lay)

## Worklist

In [6]:
from hamrep.worklist import read_worklist, check_worklist
from hamrep.readdata import read_params

wl_raw = read_worklist(WORKLIST_FILE_PATH)
params = read_params(PARAMS_FILE_PATH)

## Dilution to Concentration

Define dilution dataframe. The dataframe is indexed according plate layout, index of refference dataframe corresponds to refference of the `plate_layout_dil`.

In [7]:
# TODO: read reference value from parameters
REF_VAL_MAX = 1.7954e+10
DILUTIONS = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0]

from hamrep.sample import make_concentration
reference_conc = make_concentration(REF_VAL_MAX, DILUTIONS)

if VERBOSE_NOTEBOOK:
    display(reference_conc)

## Report generation

In [8]:
from hamrep.reportgen import gen_report_raw

reports = gen_report_raw(wl_raw, params, g_lay, reference_conc, WORKING_DIR)

['./reports/export/230801_AAV9-ELISA_sey_GN004240-053\\230801_AAV9-ELISA_1_20230801_105939.txt', './reports/export/230801_AAV9-ELISA_sey_GN004240-053\\230801_AAV9-ELISA_2_20230801_105939.txt', './reports/export/230801_AAV9-ELISA_sey_GN004240-053\\230801_AAV9-ELISA_3_20230801_105939.txt']
Processing plate 1 of 3


100%|██████████| 21/21 [00:05<00:00,  3.71it/s]


Processing plate 2 of 3


100%|██████████| 21/21 [00:05<00:00,  3.90it/s]


Processing plate 3 of 3


100%|██████████| 21/21 [00:05<00:00,  3.70it/s]


<Figure size 640x480 with 0 Axes>

In [14]:
from hamrep.mdhandling import md2docx, md2pdf, export_main_report
from hamrep.reportmd import save_md

export_main_report(reports, WORKING_DIR, cfg.pandoc_bin, cfg.reference_docx)

docxr = True
pdf = True
for report in reports:
    print('Report for plate {} saved as {}'.format(report['plate'], report['path']))
    save_md(report['path'], report['md'])

    # TODO: save unformatted results
    xlsx_file = path.splitext(report['path'])[0] + '_results.xlsx'
    report['df'].to_excel(xlsx_file)

    if docxr:
        md2docx(cfg.pandoc_bin, cfg.reference_docx, report['path'])
    if pdf:
        md2pdf(cfg.pandoc_bin, cfg.pdflatex_bin, report['path'])

print('Done.')

Generating Word ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\230801_AAV9-ELISA.docx from ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\230801_AAV9-ELISA.md
Report for plate 1 saved as ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\results_plate_1\230801_GN004240-053_-_report_plate_1.md
Generating Word ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\results_plate_1\230801_GN004240-053_-_report_plate_1.docx from ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\results_plate_1\230801_GN004240-053_-_report_plate_1.md
Generating PDF ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\results_plate_1\230801_GN004240-053_-_report_plate_1.pdf from ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\results_plate_1\230801_GN004240-053_-_report_plate_1.md
Report for plate 2 saved as ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\results_plate_2\230801_GN004240-053_-_report_plate_2.md
Generating Word ./reports/export/230801_AAV9-ELISA_sey_GN004240-053\results_plate_2

In [ ]:
from hamrep.reportmain import check_report_crc

CHECK_REPORT_CRC = True
REPORT_PLATES_CRC = [3422234280, 3350913816, 931962603]
if CHECK_REPORT_CRC:
    for report, crc in zip(reports, REPORT_PLATES_CRC):
        try:
            check_report_crc(report['md'], crc)
        except Exception as e:
            print('{} for {}'.format(e, report['path']))

Uncomment the next block to build the module

In [ ]:
! python -m build --sdist --wheel

* Creating virtualenv isolated environment...
* Installing packages in isolated environment... (setuptools >= 40.8.0, wheel)
* Getting build dependencies for sdist...
running egg_info
writing hamrep.egg-info\PKG-INFO
writing dependency_links to hamrep.egg-info\dependency_links.txt
writing top-level names to hamrep.egg-info\top_level.txt
reading manifest file 'hamrep.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'hamrep.egg-info\SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing hamrep.egg-info\PKG-INFO
writing dependency_links to hamrep.egg-info\dependency_links.txt
writing top-level names to hamrep.egg-info\top_level.txt
reading manifest file 'hamrep.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'hamrep.egg-info\SOURCES.txt'
running check
creating hamrep-0.0.2
creating hamrep-0.0.2\hamrep
creating hamrep-0.0.2\hamrep.egg-info
creating hamrep-0.0.2\test
copying files to hamrep-0.0.2...
copy